In [19]:
import json
import time

import pandas as pd

list_data = []
with open('data/category.json', 'r') as f:
    data = json.loads(f.read())

for d in data:
    children = d.pop('children')
    list_data.append(d)
    list_data += children

df_category = pd.DataFrame.from_records(list_data)
df_category.drop(axis=1, columns=['children'])
print(df_category)

s = time.time()
df_data = pd.read_csv('data/data.csv', engine='pyarrow', dtype_backend='pyarrow')
print(time.time() - s)
print(df_data.shape)


        catid  parent_catid                          name  \
0    11035567             0                   Men Clothes   
1    11035568      11035567        Jackets, Coats & Vests   
2    11035572      11035567        Suit Jackets & Blazers   
3    11035578      11035567         Hoodies & Sweatshirts   
4    11035583      11035567                         Jeans   
..        ...           ...                           ...   
309  11116487      11116484                      Handtool   
310  11116485      11116484     Large tools and equipment   
311  11116489      11116484  Electrical Circuitry & Parts   
312  11116488      11116484     Building and construction   
313  11116486      11116484                   Accessories   

                      display_name                             image  \
0                   Thời Trang Nam  687f3967b7c2fe6a134a2c11894eea4b   
1                         Áo Khoác  9251aba9f04e2f2407578c8bf1764666   
2                Áo Vest và Blazer  0f42e45eb51e8e2

In [32]:
import urllib

base_url = 'https://shopee.vn'
s = time.time()
def transform(r):
    try:
        return (r['name'],
                r['price'],
                json.loads(r['item_rating'].replace("'", '"'))['rating_star'],
                r['price'] * r['historical_sold'],
                base_url + '/' + urllib.parse.quote_plus(r['name'].replace(' ', '-') + f'-i.{r["shopid"]}.{r["itemid"]}'),
                r['itemid'], r['catid'])
    except Exception as e:
        print(r.to_dict())
        raise e

df_data[['product_name', 'product_price', 'product_rating', 'product_url', 'product_revenue', 'itemid', 'catid']] = df_data.apply(
    transform, axis=1, result_type='expand')
df_formatted_data = df_data[['product_name', 'product_price', 'product_rating', 'product_url', 'product_revenue']]
df_formatted_data.to_csv('data/formatted_data.csv', index=False)
end = time.time()
print(end - s)
print("Số lượng sản phẩm transform trong một phút", df_formatted_data.shape[0] / ((end - s) / 60))

df_full_data = df_data[['product_name', 'product_price', 'product_rating', 'product_url', 'product_revenue', 'itemid', 'catid', 'crawl_catid']].copy()
df_full_data = df_full_data.merge(df_category, how='left', left_on='crawl_catid', right_on='catid', suffixes=('', '_category'))
df_full_data

8.728617906570435
Số lượng sản phẩm transform trong một phút 1003176.0003389193


,product_name,product_price,product_rating,product_url,product_revenue,itemid,catid,crawl_catid,catid_category,parent_catid,name,display_name,image,unselected_image,selected_image,level,block_buyer_platform,children
0,Tai nghe chơi game Bluetooth không dây TWS X15...,1800000000,4.574758,29235600000000,https://shopee.vn/Tai-nghe-ch%C6%A1i-game-Blue...,16788752180,100535,11035954,11035954,0,Computer & Accessories,Máy Tính & Laptop,c3f3edfaa9f6dafc4825b77d8449999d,688db8a2d094a46c612389f24af4dfd8,c5f9534c31bcaa89f36b644454c8771b,1,None,NaN
1,[Mã SKAMSALE1023 giảm 100% đơn 99K] Loa blueto...,3900000000,4.474979,59342400000000,https://shopee.vn/%5BM%C3%A3-SKAMSALE1023-gi%E...,17210358514,100535,11035954,11035954,0,Computer & Accessories,Máy Tính & Laptop,c3f3edfaa9f6dafc4825b77d8449999d,688db8a2d094a46c612389f24af4dfd8,c5f9534c31bcaa89f36b644454c8771b,1,None,NaN
2,Chuột Logitech G102 gaming chuột chơi game có ...,1000000000,4.766753,2921000000000,https://shopee.vn/Chu%E1%BB%99t-Logitech-G102-...,11388433423,100644,11035954,11035954,0,Computer & Accessories,Máy Tính & Laptop,c3f3edfaa9f6dafc4825b77d8449999d,688db8a2d094a46c612389f24af4dfd8,c5f9534c31bcaa89f36b644454c8771b,1,None,NaN
3,bộ phát wifi tplink cục phát wifi tplink 2 râu...,2000000000,4.766738,77700000000000,https://shopee.vn/b%E1%BB%99-ph%C3%A1t-wifi-tp...,1036024433,100644,11035954,11035954,0,Computer & Accessories,Máy Tính & Laptop,c3f3edfaa9f6dafc4825b77d8449999d,688db8a2d094a46c612389f24af4dfd8,c5f9534c31bcaa89f36b644454c8771b,1,None,NaN
4,"Tai Nghe Mèo P47M. Mz47 Bluetooth Chụp Tai , H...",1000000000,4.738113,8010000000000,https://shopee.vn/Tai-Nghe-M%C3%A8o-P47M.-Mz47...,20022102716,100535,11035954,11035954,0,Computer & Accessories,Máy Tính & Laptop,c3f3edfaa9f6dafc4825b77d8449999d,688db8a2d094a46c612389f24af4dfd8,c5f9534c31bcaa89f36b644454c8771b,1,None,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145934,Găng tay da nam cảm ứng chống nước lót nỉ lông...,7900000000,4.200000,6083000000000,https://shopee.vn/G%C4%83ng-tay-da-nam-c%E1%BA...,19595767345,100009,11035627,11035627,11035567,Additional Accessories,Phụ Kiện Nam,8d2ff5f6c753fb2dc08e20c83a92fa03,None,None,2,None,NaN
145935,(HÀNG CHẤT) Mũ nồi nam trung niên [HÀNG LOẠI 1...,1000000000,4.166667,103000000000,https://shopee.vn/%28H%C3%80NG-CH%E1%BA%A4T%29...,25751233618,100009,11035627,11035627,11035567,Additional Accessories,Phụ Kiện Nam,8d2ff5f6c753fb2dc08e20c83a92fa03,None,None,2,None,NaN
145936,Găng tay nam mùa đông đi xem máy cảm ứng chống...,7900000000,4.769231,7663000000000,https://shopee.vn/G%C4%83ng-tay-nam-m%C3%B9a-%...,18893135066,100009,11035627,11035627,11035567,Additional Accessories,Phụ Kiện Nam,8d2ff5f6c753fb2dc08e20c83a92fa03,None,None,2,None,NaN
145937,Mũ len nam nữ hàn quốc kèm khăn ống qua...,5900000000,4.500000,141600000000,https://shopee.vn/Mu%CC%83-len-nam-n%C6%B0%CC%...,25454586117,100009,11035627,11035627,11035567,Additional Accessories,Phụ Kiện Nam,8d2ff5f6c753fb2dc08e20c83a92fa03,None,None,2,None,NaN


In [43]:
count = 0
for i, row in df_category[df_category['level'] == 1].iterrows():
    x = df_full_data[(df_full_data['catid_category'] == row['catid']) | (df_full_data['parent_catid'] == row['catid'])]
    count += 1
    print(f"{count}. Số lượng mặt hàng {row['display_name']} là:", x.shape[0], ". Số lượng không trùng:", x['itemid'].drop_duplicates().shape[0])
    if x['itemid'].drop_duplicates().shape[0] < 3000:
        print("----------------- WARNING ------------: số lượng mặt hàng không thỏa mãn yêu cầu đề bài")

1. Số lượng mặt hàng Thời Trang Nam là: 10741 . Số lượng không trùng: 9911
2. Số lượng mặt hàng Thời Trang Nữ là: 11000 . Số lượng không trùng: 10366
3. Số lượng mặt hàng Điện Thoại & Phụ Kiện là: 5764 . Số lượng không trùng: 5212
4. Số lượng mặt hàng Mẹ & Bé là: 7740 . Số lượng không trùng: 6913
5. Số lượng mặt hàng Thiết Bị Điện Tử là: 5573 . Số lượng không trùng: 4987
6. Số lượng mặt hàng Nhà Cửa & Đời Sống là: 8000 . Số lượng không trùng: 7345
7. Số lượng mặt hàng Máy Tính & Laptop là: 5360 . Số lượng không trùng: 4660
8. Số lượng mặt hàng Sắc Đẹp là: 6000 . Số lượng không trùng: 5361
9. Số lượng mặt hàng Máy Ảnh & Máy Quay Phim là: 3480 . Số lượng không trùng: 2872
----------------- WARNING ------------: số lượng mặt hàng không thỏa mãn yêu cầu đề bài
10. Số lượng mặt hàng Sức Khỏe là: 4116 . Số lượng không trùng: 3699
11. Số lượng mặt hàng Đồng Hồ là: 3021 . Số lượng không trùng: 2455
----------------- WARNING ------------: số lượng mặt hàng không thỏa mãn yêu cầu đề bài
12. Số l

In [39]:
# df_full_data.groupby(['itemid'], as_index=False).size()
print(df_full_data.shape)
a = df_full_data[['itemid', 'catid']].drop_duplicates()
a

(145939, 18)


,itemid,catid
0,16788752180,100535
1,17210358514,100535
2,11388433423,100644
3,1036024433,100644
4,20022102716,100535
...,...,...
145934,19595767345,100009
145935,25751233618,100009
145936,18893135066,100009
145937,25454586117,100009
